In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os


ModuleNotFoundError: No module named 'tensorflow.python'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((100, 100)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        # transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((100, 100)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [3]:
data_dir = 'Data\\data\\'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [8]:
class_names

['awake', 'sleepy']

In [4]:
model = models.mobilenet_v2(pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Modify the last layer for binary classification
model.features[0][0] = nn.Conv2d(
    in_channels=3,  # Single channel (grayscale)
    out_channels=32,
    kernel_size=(3, 3),
    stride=(2, 2),
    padding=(1, 1),
    bias=False
)
model.classifier[1] = nn.Linear(model.last_channel, 1)
model = model.to(device)


C:\Users\Kabilan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kabilan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=4):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.unsqueeze(1).float()  # Reshape for binary classification

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    preds = torch.sigmoid(outputs) > 0.5

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model


In [7]:
model = train_model(model, criterion, optimizer, dataloaders, device, num_epochs=15)


Epoch 1/15
----------
train Loss: 0.0732 Acc: 0.9744
val Loss: 0.0483 Acc: 0.9873
Epoch 2/15
----------
train Loss: 0.0480 Acc: 0.9823
val Loss: 0.0374 Acc: 0.9875
Epoch 3/15
----------
train Loss: 0.0462 Acc: 0.9843
val Loss: 0.0428 Acc: 0.9857
Epoch 4/15
----------
train Loss: 0.0431 Acc: 0.9851
val Loss: 0.0371 Acc: 0.9878
Epoch 5/15
----------
train Loss: 0.0403 Acc: 0.9857
val Loss: 0.0355 Acc: 0.9879
Epoch 6/15
----------
train Loss: 0.0374 Acc: 0.9867
val Loss: 0.0347 Acc: 0.9885
Epoch 7/15
----------
train Loss: 0.0376 Acc: 0.9862
val Loss: 0.0331 Acc: 0.9875
Epoch 8/15
----------
train Loss: 0.0352 Acc: 0.9870
val Loss: 0.0433 Acc: 0.9843
Epoch 9/15
----------
train Loss: 0.0342 Acc: 0.9876
val Loss: 0.0328 Acc: 0.9883
Epoch 10/15
----------
train Loss: 0.0330 Acc: 0.9878
val Loss: 0.0327 Acc: 0.9890
Epoch 11/15
----------
train Loss: 0.0338 Acc: 0.9878


KeyboardInterrupt: 

In [9]:
for param in model.features.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)

model = train_model(model, criterion, optimizer, dataloaders, device, num_epochs=10)


Epoch 1/10
----------
train Loss: 0.0282 Acc: 0.9897
val Loss: 0.0286 Acc: 0.9896
Epoch 2/10
----------
train Loss: 0.0257 Acc: 0.9904
val Loss: 0.0281 Acc: 0.9904
Epoch 3/10
----------
train Loss: 0.0259 Acc: 0.9906
val Loss: 0.0277 Acc: 0.9905
Epoch 4/10
----------
train Loss: 0.0255 Acc: 0.9906
val Loss: 0.0275 Acc: 0.9907
Epoch 5/10
----------
train Loss: 0.0252 Acc: 0.9909
val Loss: 0.0273 Acc: 0.9903
Epoch 6/10
----------
train Loss: 0.0238 Acc: 0.9912
val Loss: 0.0273 Acc: 0.9907
Epoch 7/10
----------
train Loss: 0.0247 Acc: 0.9910
val Loss: 0.0270 Acc: 0.9909
Epoch 8/10
----------
train Loss: 0.0236 Acc: 0.9909
val Loss: 0.0267 Acc: 0.9908
Epoch 9/10
----------
train Loss: 0.0231 Acc: 0.9916
val Loss: 0.0265 Acc: 0.9908
Epoch 10/10
----------
train Loss: 0.0240 Acc: 0.9911
val Loss: 0.0265 Acc: 0.9910


In [10]:
torch.save(model.state_dict(), 'mobilenet_binary_classifier1.pth')


In [12]:
def evaluate_model(model, dataloader, device):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()

            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloader.dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')


In [13]:
evaluate_model(model, dataloaders['val'], device)

Validation Accuracy: 0.9888


In [31]:
import torch

# Path to the saved model
model_path = "mobilenet_binary_classifier1.pth"

# Load MobileNet model
from torchvision import models
mobilenet = models.mobilenet_v2(pretrained=False)

# Modify for binary classification
mobilenet.classifier[1] = torch.nn.Linear(mobilenet.last_channel, 1)

# Load the saved weights
mobilenet.load_state_dict(torch.load(model_path))
mobilenet.eval()  # Set model to evaluation mode


C:\Users\Kabilan\AppData\Local\Temp\ipykernel_35444\2842321566.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mobilenet.load_state_dict(torch.load(model_path))


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [34]:
from torchvision import transforms
from PIL import Image

# Define the same transformations used during training
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to grayscale with 3 channels
    transforms.Resize((224, 224)),               # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize for 3 channels
])

# Load the test image
test_image_path = "Data\\data\\test\\awake\\s0001_01846_0_0_1_0_0_01.png"  # Path to your test image
test_image = Image.open(test_image_path)

# Apply transformations
input_image = data_transforms(test_image).unsqueeze(0)  # Add batch dimension


In [35]:
# Move model and input to the appropriate device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet = mobilenet.to(device)
input_image = input_image.to(device)

# Perform inference
with torch.no_grad():
    outputs = mobilenet(input_image)
    predicted = torch.sigmoid(outputs) > 0.5
    # _, predicted = torch.max(outputs, 1)  # Get the class index with the highest score

# Map prediction to class labels
class_labels = {0: "Closed", 1: "Open"}  # Example labels
predicted_label = class_labels[predicted.item()]

print(f"Prediction: {predicted_label}")


Prediction: Open
